In [3]:
#from bokeh.models.widgets import Panel, Tabs
#from bokeh.io import output_file, show
#from bokeh.plotting import figure

#output_file("slider.html")

##p1.circle([1, 2, 3, 4, 5], [6, 7, 2, 4, 5], size=20, color="navy", alpha=0.5)
#tab1 = Panel(child=p1, title="circle")

#p2 = figure(plot_width=300, plot_height=300)
#p2.line([1, 2, 3, 4, 5], [6, 7, 2, 4, 5], line_width=3, color="navy", alpha=0.5)
#tab2 = Panel(child=p2, title="line")

#tabs = Tabs(tabs=[ tab1, tab2 ])

#show(tabs)'''

In [1]:
import numpy as np

from bokeh.plotting import figure ,show,output_file
import pandas as pd
from bokeh.models import Slider, Select, TextInput, Panel, Tabs
from bokeh.models import HoverTool, BoxSelectTool,CustomJS, ColumnDataSource

from bokeh.layouts import widgetbox, layout,row,column


output_file("budget.html")

SRECNA2 =pd.read_excel('SISD.xlsx')
SRECNA3=SRECNA2.dropna(axis=0,how='all')  #Drops rows with all NaN values
Indexes=list(SRECNA3.index.values)
SRECNA3=SRECNA3.drop([Indexes[8],Indexes[20],Indexes[19],Indexes[18],Indexes[17],Indexes[16],Indexes[15]],axis=0)
Depreciation =pd.read_excel('Income Statement Data v.1.xlsx')
Depreciation=Depreciation.dropna(axis=0,how='all')
indexes=list(Depreciation.index.values)
depreciation=Depreciation.loc[[indexes[17],indexes[19]],:]
depreciation=depreciation.values[:,:-1]
newValues=SRECNA3.values[:,:]
nextYears=np.zeros((newValues.shape[0],newValues.shape[1]))
Svalues=np.hstack((newValues,nextYears))
Svalues=np.vstack((Svalues,depreciation))

class Operator:
    """An operator operates on the global set of data  scenario."""

    def __init__(self,data):
        """Create an operator with a given section with the given exit.

        name -- A string; the name of this Place.
        exit -- The Place reached by exiting this Place (may be None).
        """
        self.base_levels=np.array([1.5,2.0,0.0,0.0,2.0,2.0,0.0,1.5,1.5,1.5,0.0,1.5,0.0,0.0])
        self.data=np.copy(data)
        self.years_update=data.shape[1]
    def leveler(self, section, assumption):
        i=4
        while(i < self.years_update):    
            newEntry=self.data[section,i-1]*(1.0+assumption)
            self.data[section,i]=newEntry
            i+=1
        totalRevs=np.sum(self.data[0:8,:],axis=0)
        totalExps=np.sum(self.data[8:-2,:],axis=0)
        depreciation=np.sum(self.data[-2:,:],axis=0)
        new_data=totalRevs-totalExps-depreciation
        return new_data/1000
    #Creates case using base_levels for all operators
    def basecase(self):
        section=0
        initial=self.leveler(0,0) # initialized the first 4 values
        for i in self.base_levels:
            initial[4:]=self.leveler(section,i)[4:]
            section+=1
        return initial
            
years=np.linspace(2014,2020,8) # 4 base years and 3 years being operating one. 7 operating years
operator1=Operator(Svalues)
operator2=Operator(Svalues)
y1=operator1.basecase()
y2=operator2.basecase()
source1 = ColumnDataSource(data=dict(x=years, y=y1))

source2 = ColumnDataSource(data=dict(x=years, y=y2))


hover = HoverTool(
        tooltips=[
            ("Index", "$index"),
            ("Net Income", "$y")
        ]
    )

TOOLS = [BoxSelectTool(), hover, 'crosshair']
model = figure(title="Budget Model", plot_height=600, plot_width=1000,tools=TOOLS) 
r = model.line('x', 'y', source=source1, color="#2222aa", line_width=2,line_dash=[4, 4], legend ="Scenario")
base=model.line('x', 'y', source=source2, line_width=2,legend="Base Case")
#Change background color
model.background_fill_color = "white"
# change just some things about the x-axes
model.xaxis.axis_label = "Years"
model.xaxis.axis_line_width = 2
model.xaxis.axis_line_color = "Black"

# change just some things about the y-axes

model.yaxis.axis_label = "Net Income (Millions)"
model.yaxis.major_label_text_color = "Black"
model.yaxis.major_label_orientation = "horizontal"

# change just some things about the x-grid
model.xgrid.grid_line_color = "Black"
model.xgrid.grid_line_alpha = 0.3
# change just some things about the y-grid
model.ygrid.grid_line_alpha = 0.1
model.ygrid.grid_line_color = "Black"
#Change the outline of the plot box
model.outline_line_width = 10
model.outline_line_alpha = 0.3
model.outline_line_color = "navy"
#Interacting widgets for Revenues
def update_tuition(source=source1):
    value = cb_obj.value
    data = source.data
    y=data['y']
    y=operator1.leveler(0, value)
    #source.data['y'] = new_data
    source.trigger('change')
    
    
def update_state(source=source1):
    value = cb_obj.value
    new_data=operator1.leveler(1, value)
    source.data['y'] = new_data
    source.trigger('change')


def update_pell_grants(source=source1):
    value = cb_obj.value
    new_data=operator1.leveler(2, value)
    source.data['y'] = new_data
    source.trigger('change')


def update_contracts(source=source1):
    value = cb_obj.value
    new_data=operator1.leveler(3, value)
    source.data['y'] = new_data
    source.trigger('change')
    
def update_educational_activities(source= source1):
    value = cb_obj.value
    new_data=operator1.leveler(4, value)
    source.data['y'] = new_data
    source.trigger('change')


def update_private_gifts(source=source1):
    value = cb_obj.value
    new_data=operator1.leveler(5,value)
    source.data['y'] = new_data
    source.trigger('change')

def update_investment_income(source=source1):
    value = cb_obj.value
    new_data=operator1.leveler(6, value)
    source.data['y'] = new_data
    source.trigger('change')

def update_otherr(source=source1):
    value = cb_obj.value
    new_data=operator1.leveler(7,value )
    source.data['y'] = new_data
    source.trigger('change')


    
#Interacting widgets for expenses

def update_salaries(source=source1, window=None):
    value = cb_obj.value
    new_data=operator1.leveler(8, value)
    source.data['y'] = new_data
    source.trigger('change')
    
def update_benefits(source=source1, window=None):
    value = cb_obj.value
    new_data=operator1.leveler(9, value)
    source.data['y'] = new_data
    source.trigger('change')


def update_scholarships(source=source1, window=None):
    value = cb_obj.value
    new_data=operator1.leveler(10, value)
    source.data['y'] = new_data
    source.trigger('change')

    
def update_utilities(source=source1, window=None):
    value = cb_obj.value
    new_data=operator1.leveler(11, value)
    source.data['y'] = new_data
    source.trigger('change')

    
def update_supplies(source=source1, window=None):
    value = cb_obj.value
    new_data=operator1.leveler(12, value)
    source.data['y'] = new_data
    source.trigger('change')
    
def update_othere(source=source1, window=None):
    value = cb_obj.value
    new_data=operator1.leveler(13, value)
    source.data['y'] = new_data
    source.trigger('change')

#Revenues Levelers
Tuition = Slider(title="Tuition", 
                 start=-1.5, end=4,step=1.5,value=1.5,
                 callback=CustomJS.from_py_func(update_tuition))

State = Slider(title="State",
               start=-2.0, end=6.0,step=2.0,value=2.0,
               callback_policy='mouseup', callback=CustomJS.from_py_func(update_state))
Pell_grants = Slider(title="Pell Grants",
                     start=-5.0, end=5.0,step=2.5,value=0.0,
                     callback_policy='mouseup', callback=CustomJS.from_py_func(update_pell_grants))
Contracts = Slider(title="Contracts",
                   start=-5.0, end=5.0,step=2.5,value=0.0,
                   callback_policy='mouseup', callback=CustomJS.from_py_func(update_contracts))
Educational_activities = Slider(title="Educational Activities",
                                start=-2.0, end=6.0,step=2.0, value=2.0,
                                callback_policy='mouseup',callback=CustomJS.from_py_func(update_educational_activities))
Private_gifts = Slider(title="Private Gifts",
                       start=-2.0, end=6.0,step=2.0,value=2.0,
                       callback_policy='mouseup', callback=CustomJS.from_py_func(update_private_gifts))
Investment_income = Slider(title="Investment income",
                           start=-10.0, end=10.0,step=5.0,value=0.0,
                           callback_policy='mouseup',callback=CustomJS.from_py_func(update_investment_income))
Otherr = Slider(title="Other",
                start=-1.5, end=4.0,step=1.5,value=1.5,
                callback_policy='mouseup', callback=CustomJS.from_py_func(update_otherr))

#Expenses Levelers
Salaries = Slider(title="Salaries",
                  start=-1.5, end=4.0,step=1.5,value=1.5,
                  callback_policy='mouseup',callback=CustomJS.from_py_func(update_salaries))
Benefits = Slider(title="Benefits",
                  start=-1.5, end=4.0,step=1.5,value=1.5,
                  callback_policy='mouseup',callback=CustomJS.from_py_func(update_benefits))
Scholarships = Slider(title="Scholarships and Fellowships",
                      start=-5.0, end=5.0,step=2.5,value=0.0,
                      callback_policy='mouseup',callback=CustomJS.from_py_func(update_scholarships))
Utilities= Slider(title="Utilities",
                  start=-1.5, end=4.0,step=1.5,value=1.5, 
                  callback_policy='mouseup',callback=CustomJS.from_py_func(update_utilities))
Supplies= Slider(title="Supplies",start=-5.0, end=5.0,step=2.5,value=0.0,
                 callback_policy='mouseup',callback=CustomJS.from_py_func(update_supplies))
Othere= Slider(title="Other",
               start=-5.0, end=5.0,step=2.5,value=0.0,
               callback_policy='mouseup',callback=CustomJS.from_py_func(update_othere))



#revenues=[Tuition,State, Pell_grants,Contracts,Educational_activities,Private_gifts,Investment_income,Otherr]
#expenses=[Salaries,Benefits,Scholarships,Utilities,Supplies,Othere]
#Revenues= widgetbox(revenues)

#Expenses= widgetbox(expenses)
#tab1 = Panel(child=Revenues, title="Revenues")
#tab2 = Panel(child=Expenses, title="Expenses")

#tabs = Tabs(tabs=[ Revenues, Expenses ])
#Revenues =column(Tuition,State, Pell_grants,Contracts,Educational_activities,Private_gifts,Investment_income,Otherr)
l=row(Tuition,model)
show(l)


    

In [2]:
from bokeh.layouts import column
from bokeh.models import CustomJS, ColumnDataSource, Slider
from bokeh.plotting import Figure, output_file, show

output_file("callback.html")

x = [x*0.005 for x in range(0, 200)]
y = x

source = ColumnDataSource(data=dict(x=x, y=y))

plot = Figure(plot_width=400, plot_height=400)
plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

def callback(source=source, window=None):
    data = source.data
    f = cb_obj.value
    x, y = data['x'], data['y']
    for i in range(len(x)):
        y[i] = window.Math.pow(x[i], f)
    source.trigger('change')

slider = Slider(start=0.1, end=4, value=1, step=.1, title="power",
                callback=CustomJS.from_py_func(callback))

layout = column(slider, plot)

show(layout)

INFO:bokeh.core.state:Session output file 'callback.html' already exists, will be overwritten.
